In [2]:
import urllib.request
import json
import os
import ssl
from dotenv import load_dotenv

load_dotenv()

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
# Example user preferences and available bubble teas
data = {
        "user_preferences": [{
            "user_id": 5,
            "fruity_user": 1,
            "milky_user": 0,
            "with_tea_user": 1,
            "refreshing_user": 1,
            "fragrant_user": 0,
            "adventurous_user": 1,
            "cold_user": 1
        }],
        "available_bubble_teas": [
            {
                "drink_id": 200,
                "fruity_drink": 1,
                "milky_drink": 0,
                "with_tea_drink": 1,
                "refreshing_drink": 1,
                "fragrant_drink": 0,
                "cold_drink": 1,
                "distance": 2.5,
                "popularity": 85
            },
            {
                "drink_id": 201,
                "fruity_drink": 0,
                "milky_drink": 1,
                "with_tea_drink": 1,
                "refreshing_drink": 0,
                "fragrant_drink": 1,
                "cold_drink": 1,
                "distance": 3.0,
                "popularity": 90
            },
            {
                "drink_id": 202,
                "fruity_drink": 1,
                "milky_drink": 1,
                "with_tea_drink": 0,
                "refreshing_drink": 1,
                "fragrant_drink": 0,
                "cold_drink": 0,
                "distance": 1.5,
                "popularity": 75
            }
        ]
    }

body = str.encode(json.dumps(data))

url = 'https://bobafy-endpoint07212348548964.eastus2.inference.ml.azure.com/score'
# Replace this with the primary/secondary key, AMLToken, or Microsoft Entra ID token for the endpoint
api_key = os.getenv("MODEL_API_KEY")
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'blue' }

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'[{"user_id": 5, "drink_id": 200, "fruity_user": 1, "milky_user": 0, "with_tea_user": 1, "refreshing_user": 1, "fragrant_user": 0, "adventurous_user": 1, "cold_user": 1, "fruity_drink": 1, "milky_drink": 0, "with_tea_drink": 1, "refreshing_drink": 1, "fragrant_drink": 0, "cold_drink": 1, "distance": 2.5, "popularity": 85, "score": 0.950219452381134}, {"user_id": 5, "drink_id": 202, "fruity_user": 1, "milky_user": 0, "with_tea_user": 1, "refreshing_user": 1, "fragrant_user": 0, "adventurous_user": 1, "cold_user": 1, "fruity_drink": 1, "milky_drink": 1, "with_tea_drink": 0, "refreshing_drink": 1, "fragrant_drink": 0, "cold_drink": 0, "distance": 1.5, "popularity": 75, "score": -0.139688178896904}, {"user_id": 5, "drink_id": 201, "fruity_user": 1, "milky_user": 0, "with_tea_user": 1, "refreshing_user": 1, "fragrant_user": 0, "adventurous_user": 1, "cold_user": 1, "fruity_drink": 0, "milky_drink": 1, "with_tea_drink": 1, "refreshing_drink": 0, "fragrant_drink": 1, "cold_drink": 1, "distan